In [1]:
import xarray as xr
import xskillscore as xs
import os
import numpy as np
import sys
import json
from datetime import datetime
from S2S_on_SFNO.Models.provenance import system_monitor
from multiprocessing import Pool, active_children
from time import sleep
import psutil
import gc
from time import time
import matplotlib.pyplot as plt


In [2]:

year = 2019
variable_index = 0
cluster = True
variables = [
    ('10m_u_component_of_wind', '10u'),
    ('10m_v_component_of_wind','10v'),
    ('2m_temperature','2t'),
    ('total_column_water_vapour','tcwv')
]
variable = variables[variable_index][0]
dataset_var = variables[variable_index][1]

save_interval = 6
if cluster:
    save_path = "/mnt/qb/work2/goswami0/gkd965/climate/skillscores"
    basePath = "/mnt/qb/work2/goswami0/gkd965/"
    dataPath = os.path.join("/mnt/qb/goswami/data/era5","single_pressure_level",variable,variable+"_{}.nc")
else:
    save_path = "/mnt/V/Master/climate/skillscores"
    basePath = "/mnt/V/Master"
    dataPath = os.path.join("/mnt/V/Master/data",variable,variable+"_{}.nc") # no single_pressure_level dir on local machine

mean_files = {
    '10m_u_component_of_wind':'hourofyear_mean_for_10m_u_component_of_wind_from_1979_to_2017created_20240123-0404.nc',
    '10m_v_component_of_wind':'hourofyear_mean_for_10m_v_component_of_wind_from_1979_to_2019created_20231211-1339.nc',
    '2':'hourofyear_mean_for_2m_temperature_from_1979_to_2017created_20240123-0343.nc',
    'total_column_water_vapour':'hourofyear_mean_for_total_column_water_vapour_from_1979_to_2017created_20240123-0415.nc'

}



In [3]:

date_string = datetime.now().strftime("%Y%m%d-%H%M")
mean_file = os.path.join(basePath,"climate",mean_files[variable])
model_file_sfno = os.path.join(basePath,'outputs/sfno','leadtime_8760_startDate_201901010_createdOn_20240123T0337/leadtime_8760_startDate_201901010_createdOn_20240123T0337_step_{}.nc')
model_file_fcn = os.path.join(basePath,'outputs/fourcastnet','leadtime_8760_startDate_201901010_createdOn_20240123T0408/leadtime_8760_startDate_201901010_createdOn_20240123T0408_step_{}.nc')
save_file = os.path.join(basePath,'')


ds_ref_alltimes  = xr.open_dataset(mean_file,chunks={'time':1})#.to_array().squeeze()[:min_step*6:6]
g_truth = xr.open_dataset(dataPath.format(year))#.to_array().squeeze()[:min_step*6:6]

end = 128


num_nans = {"sfno":[],"fcn":[]}
skill_scores = {"sfno":[],"fcn":[]}

/opt/conda/envs/model/lib/python3.11/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.30.0
  warnings.warn(


In [4]:

model_file_fcn

'/mnt/qb/work2/goswami0/gkd965/outputs/fourcastnet/leadtime_8760_startDate_201901010_createdOn_20240123T0408/*'

In [7]:
ds_sfno = xr.open_mfdataset(model_file_sfno)[dataset_var]

In [16]:
# ds_sfno[dataset_var].squeeze()

<xarray.DataArray '10u' (step: 128, latitude: 721, longitude: 1440)>
dask.array<concatenate, shape=(128, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * step       (step) timedelta64[ns] 0 days 06:00:00 ... 32 days 00:00:00

In [9]:
ds_ref  = xr.open_dataset(mean_file,chunks={'time':1})

In [10]:
ds_ref

<xarray.Dataset>
Dimensions:                        (longitude: 1440, latitude: 721, time: 8760)
Coordinates:
  * longitude                      (longitude) float32 0.0 0.25 ... 359.5 359.8
  * latitude                       (latitude) float32 90.0 89.75 ... -90.0
    variable                       <U3 ...
  * time                           (time) int64 0 1 2 3 ... 8756 8757 8758 8759
Data variables:
    __xarray_dataarray_variable__  (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [6]:
ds_ref_a = ds_ref[data_var]

NameError: name 'ds_ref' is not defined

<xarray.DataArray (variable: 1, time: 128, latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(1, 128, 721, 1440), dtype=float32, chunksize=(1, 1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * variable   (variable) object '__xarray_dataarray_variable__'
  * time       (time) int64 0 6 12 18 24 30 36 ... 726 732 738 744 750 756 762

In [ ]:
ds_ref_a_a = ds_ref_a[:min_step*6:6]

In [4]:

ds_fcn = xr.open_mfdataset(model_file_fcn,chunks={'step':1})

/opt/conda/envs/model/lib/python3.11/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.30.0
  warnings.warn(


In [15]:
ds_fcn_a = ds_fcn[dataset_var]

In [34]:
ds_sfno.shape[ds_sfno.dims.index('step')]

128

In [11]:

min_step = np.min([ds_sfno.dims["step"],ds_fcn.dims["step"]])

TypeError: tuple indices must be integers or slices, not str

In [ ]:
ds_ref_a[:,:768:6,:,:]

In [16]:
min_step = np.min([ds_sfno.shape[ds_sfno.dims.index('step')],ds_fcn_a.shape[ds_fcn_a.dims.index('step')]])

In [17]:
min_step

128

In [4]:
idx = 0
s = (idx+1)*6
ds_sfno = xr.open_dataset(model_file_sfno.format(s))[dataset_var] # needs 4 min
ds_fcn = xr.open_mfdataset(model_file_fcn.format(s))[dataset_var] 
ds_ref = ds_ref_alltimes.sel(time=idx).to_array().squeeze()
truth = g_truth.sel(time=np.datetime64(str(year)+'-01-01T00:00:00.000000000') + np.timedelta64(s, 'h')).to_array().squeeze()


nans_sfno = np.isnan(ds_sfno)
nans_fcn  = np.isnan(ds_fcn)
num_nans["sfno"].append(nans_sfno.sum())
num_nans["fcn"].append(nans_fcn.sum())

rmse_sfno = xs.rmse(ds_sfno,truth,dim=["latitude","longitude"],skipna=True)
rmse_fcn  = xs.rmse(ds_fcn ,truth,dim=["latitude","longitude"],skipna=True)
rmse_ref  = xs.rmse(ds_ref,truth,dim=["latitude","longitude"])

skill_score_sfno = 1 - rmse_sfno/rmse_ref
skill_score_fcn  = 1 - rmse_fcn/rmse_ref
skill_scores["sfno"].append(skill_score_sfno)
skill_scores["fcn"].append(skill_score_fcn)

rmse_sfno_globe = xs.rmse(ds_sfno,truth,dim=[],skipna=True)
rmse_fcn_globe  = xs.rmse(ds_fcn ,truth,dim=[],skipna=True)

In [5]:
sfno_d = rmse_sfno_globe.to_dataset(name="rmse")

In [7]:
sfno_d.assign_coords(step=[6])

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, step: 1)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    time       datetime64[ns] 2019-01-01T06:00:00
    variable   <U3 'u10'
  * step       (step) int64 6
Data variables:
    rmse       (latitude, longitude) float32 0.1168 0.1168 ... 1.809 1.809

In [ ]:

for idx in range(end):
    s = (idx+1)*6
    ds_sfno = xr.open_dataset(model_file_sfno.format(s))[dataset_var] # needs 4 min
    ds_fcn = xr.open_mfdataset(model_file_fcn.format(s))[dataset_var] 
    ds_ref = ds_ref_alltimes.sel(time=idx).to_array().squeeze()
    truth = g_truth.sel(time=np.datetime64(str(year)+'-01-01T00:00:00.000000000') + np.timedelta64(s, 'h')).to_array().squeeze()


    nans_sfno = np.isnan(ds_sfno)
    nans_fcn  = np.isnan(ds_fcn)
    num_nans["sfno"].append(nans_sfno.sum())
    num_nans["fcn"].append(nans_fcn.sum())

    rmse_sfno = xs.rmse(ds_sfno,truth,dim=["latitude","longitude"],skipna=True)
    rmse_fcn  = xs.rmse(ds_fcn ,truth,dim=["latitude","longitude"],skipna=True)
    rmse_ref  = xs.rmse(ds_ref,truth,dim=["latitude","longitude"])

    skill_score_sfno = 1 - rmse_sfno/rmse_ref
    skill_score_fcn  = 1 - rmse_fcn/rmse_ref
    skill_scores["sfno"].append(skill_score_sfno)
    skill_scores["fcn"].append(skill_score_fcn)

    rmse_sfno_globe = xs.rmse(ds_sfno,truth,dim=[],skipna=True)
    rmse_fcn_globe  = xs.rmse(ds_fcn ,truth,dim=[],skipna=True)

    rmse_sfno_globe.to_dataset(name="rmse").save(os.path.join(save_path,"rmse_global_sfno_"+variable+"_hr_"+str(s)+"_"+date_string+".nc"))
    rmse_fcn_globe.to_dataset().save(os.path.join(save_path,"rmse_global_fcn_"+variable+"_hr_"+str(s)+"_"+date_string+".nc"))

    if idx%save_interval == 0:
        print("saving skill scores to "+save_path,flush=True)
        np.save(os.path.join(save_path,"rmse_sfno_"+variable+"_"+date_string+".npz"),skill_scores['sfno'])
        np.save(os.path.join(save_path,"rmse_fcn_"+variable+"_"+date_string+".npz"),skill_scores['fcn'])
        np.save(os.path.join(save_path,"nans_sfno_"+variable+"_"+date_string+".npz"),num_nans['sfno'])
        np.save(os.path.join(save_path,"nans_fcn_"+variable+"_"+date_string+".npz"),num_nans['fcn'])

np.save(os.path.join(save_path,"rmse_sfno_"+variable+"_"+date_string+"_fin.npz"),skill_scores['sfno'])
np.save(os.path.join(save_path,"rmse_fcn_"+variable+"_"+date_string+"_fin.npz"),skill_scores['fcn'])
np.save(os.path.join(save_path,"nans_sfno_"+variable+"_"+date_string+"_fin.npz"),num_nans['sfno'])
np.save(os.path.join(save_path,"nans_fcn_"+variable+"_"+date_string+"_fin.npz"),num_nans['fcn'])



In [2]:
path = "/mnt/ssd2/Master/S2S_on_SFNO/outputs/sfno/"
file = "20231108.grib"

In [25]:

# file = "SFNO_HalfYearForecast_20231116-1441.grib"
ds_sfno_chunks = xr.open_dataset(os.path.join(path,file), engine="cfgrib", chunks={"step": 4})
ds_sfno = xr.open_dataset(os.path.join(path,file), engine="cfgrib")

skipping variable: paramId==228246 shortName='u100'
Traceback (most recent call last):
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=10.0) new_value=Variable(dimensions=(), data=100.0)
skipping variable: paramId==228247 shortName='v100'
Traceback (most recent call last):
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.data

In [35]:
def func(da):
    print(da.sizes)
    return da.step

mapped = xr.map_blocks(func, ds_sfno_chunks)

Frozen({'step': 0, 'latitude': 0, 'longitude': 0, 'isobaricInhPa': 0})


In [37]:
mapped.load()

Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})
Frozen({'step': 4, 'latitude': 721, 'longitude': 1440, 'isobaricInhPa': 13})


<xarray.DataArray 'step' (step: 40)>
array([ 21600000000000,  43200000000000,  64800000000000,  86400000000000,
       108000000000000, 129600000000000, 151200000000000, 172800000000000,
       194400000000000, 216000000000000, 237600000000000, 259200000000000,
       280800000000000, 302400000000000, 324000000000000, 345600000000000,
       367200000000000, 388800000000000, 410400000000000, 432000000000000,
       453600000000000, 475200000000000, 496800000000000, 518400000000000,
       540000000000000, 561600000000000, 583200000000000, 604800000000000,
       626400000000000, 648000000000000, 669600000000000, 691200000000000,
       712800000000000, 734400000000000, 756000000000000, 777600000000000,
       799200000000000, 820800000000000, 842400000000000, 864000000000000],
      dtype='timedelta64[ns]')
Coordinates:
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 10 days 00:...
    time               datetime64[ns] 2023-01-10
    heightAboveGround  float64 10.0
    valid_time         (step) datetime64[ns] 2023-01-10T06:00:00 ... 2023-01-20
    surface            float64 0.0
    meanSea            float64 0.0
    entireAtmosphere   float64 0.0
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period

In [ ]:
def mean_parallel(x):
    return x.mean(dim="step")

xr.apply_ufunc(
        mean_parallel,
        ds_sfno_chunks,
        # input_core_dims=[[dim], [dim]],
        dask="parallelized",
        # output_dtypes=[float],
    )

In [26]:
ds_mean = ds_sfno_chunks.mean(dim="step")

In [27]:
ds_mean.compute()

<xarray.Dataset>
Dimensions:            (latitude: 721, longitude: 1440, isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 2023-01-10
    heightAboveGround  float64 10.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    surface            float64 0.0
    meanSea            float64 0.0
    entireAtmosphere   float64 0.0
  * isobaricInhPa      (isobaricInhPa) float64 1e+03 925.0 850.0 ... 100.0 50.0
Data variables:
    u10                (latitude, longitude) float32 0.8723 0.8723 ... 6.199
    v10                (latitude, longitude) float32 -1.372 -1.372 ... 2.858
    sp                 (latitude, longitude) float32 1.018e+05 ... 7.45e+04
    msl                (latitude, longitude) float32 1.021e+05 ... 9.779e+04
    tcwv               (latitude, longitude) float32 2.093 2.093 ... -24.02
    u                  (isobaricInhPa, latitude, longitude) float32 0.3506 .....
    v                  (isobaricInhPa, latitude, longitude) float32 -0.4631 ....
    z                  (isobaricInhPa, latitude, longitude) float32 1.478e+03...
    t                  (isobaricInhPa, latitude, longitude) float32 248.2 ......
    r                  (isobaricInhPa, latitude, longitude) float32 100.3 ......

In [20]:
xr.apply_ufunc(
        np.mean,
        ds_sfno,
        dask="parallelized",
    )


<xarray.Dataset>
Dimensions:            (step: 40, latitude: 721, longitude: 1440,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 2023-01-10
  * step               (step) timedelta64[ns] 0 days 06:00:00 ... 10 days 00:...
    heightAboveGround  float64 10.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time         (step) datetime64[ns] dask.array<chunksize=(4,), meta=np.ndarray>
    surface            float64 0.0
    meanSea            float64 0.0
    entireAtmosphere   float64 0.0
  * isobaricInhPa      (isobaricInhPa) float64 1e+03 925.0 850.0 ... 100.0 50.0
Data variables:
    u10                (step, latitude, longitude) float32 dask.array<chunksize=(4, 721, 1440), meta=np.ndarray>
    v10                (step, latitude, longitude) float32 dask.array<chunksize=(4, 721, 1440), meta=np.ndarray>
    sp                 (step, latitude, longitude) float32 dask.array<chunksize=(4, 721, 1440), meta=np.ndarray>
    msl                (step, latitude, longitude) float32 dask.array<chunksize=(4, 721, 1440), meta=np.ndarray>
    tcwv               (step, latitude, longitude) float32 dask.array<chunksize=(4, 721, 1440), meta=np.ndarray>
    u                  (step, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(4, 13, 721, 1440), meta=np.ndarray>
    v                  (step, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(4, 13, 721, 1440), meta=np.ndarray>
    z                  (step, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(4, 13, 721, 1440), meta=np.ndarray>
    t                  (step, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(4, 13, 721, 1440), meta=np.ndarray>
    r                  (step, isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(4, 13, 721, 1440), meta=np.ndarray>

In [ ]:
ds = (mv.read("ERA5_levels.grib")).to_dataset()

In [3]:
ds_sfno.info()

xarray.Dataset {
dimensions:
	step = 40 ;
	latitude = 721 ;
	longitude = 1440 ;
	isobaricInhPa = 13 ;

variables:
	datetime64[ns] time() ;
		time:long_name = initial time of forecast ;
		time:standard_name = forecast_reference_time ;
	timedelta64[ns] step(step) ;
		step:long_name = time since forecast_reference_time ;
		step:standard_name = forecast_period ;
	float64 heightAboveGround() ;
		heightAboveGround:long_name = height above the surface ;
		heightAboveGround:units = m ;
		heightAboveGround:positive = up ;
		heightAboveGround:standard_name = height ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	datetime64[ns] valid_time(step) ;
		valid_time:standard_name = time ;
		valid_time:long_name = time ;
	float32 u

In [12]:
index = list(range(0,40,4))
step = [np.timedelta64(21600000000000*i,'ns') for i in index]
print(step)
red = ds_sfno.groupby_bins("step",step)

[numpy.timedelta64(0,'ns'), numpy.timedelta64(86400000000000,'ns'), numpy.timedelta64(172800000000000,'ns'), numpy.timedelta64(259200000000000,'ns'), numpy.timedelta64(345600000000000,'ns'), numpy.timedelta64(432000000000000,'ns'), numpy.timedelta64(518400000000000,'ns'), numpy.timedelta64(604800000000000,'ns'), numpy.timedelta64(691200000000000,'ns'), numpy.timedelta64(777600000000000,'ns')]


In [31]:
red = ds_sfno.groupby("step.days").mean(dim="step")

In [32]:
red

<xarray.Dataset>
Dimensions:            (latitude: 721, longitude: 1440, days: 11,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 2023-01-10
    heightAboveGround  float64 10.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    surface            float64 0.0
    meanSea            float64 0.0
    entireAtmosphere   float64 0.0
  * isobaricInhPa      (isobaricInhPa) float64 1e+03 925.0 850.0 ... 100.0 50.0
  * days               (days) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    u10                (days, latitude, longitude) float32 4.465 4.465 ... 5.649
    v10                (days, latitude, longitude) float32 -1.335 ... 1.146
    sp                 (days, latitude, longitude) float32 1.015e+05 ... 7.45...
    msl                (days, latitude, longitude) float32 1.019e+05 ... 9.78...
    tcwv               (days, latitude, longitude) float32 0.2621 ... -27.89
    u                  (days, isobaricInhPa, latitude, longitude) float32 2.9...
    v                  (days, isobaricInhPa, latitude, longitude) float32 -0....
    z                  (days, isobaricInhPa, latitude, longitude) float32 1.3...
    t                  (days, isobaricInhPa, latitude, longitude) float32 248...
    r                  (days, isobaricInhPa, latitude, longitude) float32 98....
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-11-17T15:38 GRIB to CDM+CF via cfgrib-0.9.1...

In [13]:
red.groups

{Interval(Timedelta('0 days 00:00:00'), Timedelta('1 days 00:00:00'), closed='right'): [0,
  1,
  2,
  3],
 Interval(Timedelta('1 days 00:00:00'), Timedelta('2 days 00:00:00'), closed='right'): [4,
  5,
  6,
  7],
 Interval(Timedelta('2 days 00:00:00'), Timedelta('3 days 00:00:00'), closed='right'): [8,
  9,
  10,
  11],
 Interval(Timedelta('3 days 00:00:00'), Timedelta('4 days 00:00:00'), closed='right'): [12,
  13,
  14,
  15],
 Interval(Timedelta('4 days 00:00:00'), Timedelta('5 days 00:00:00'), closed='right'): [16,
  17,
  18,
  19],
 Interval(Timedelta('5 days 00:00:00'), Timedelta('6 days 00:00:00'), closed='right'): [20,
  21,
  22,
  23],
 Interval(Timedelta('6 days 00:00:00'), Timedelta('7 days 00:00:00'), closed='right'): [24,
  25,
  26,
  27],
 Interval(Timedelta('7 days 00:00:00'), Timedelta('8 days 00:00:00'), closed='right'): [28,
  29,
  30,
  31],
 Interval(Timedelta('8 days 00:00:00'), Timedelta('9 days 00:00:00'), closed='right'): [32,
  33,
  34,
  35]}

In [14]:
red_mean = red.mean()

<xarray.Dataset>
Dimensions:            (latitude: 721, longitude: 1440, step_bins: 9,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 2023-01-10
    heightAboveGround  float64 10.0
  * latitude           (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude          (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    surface            float64 0.0
    meanSea            float64 0.0
    entireAtmosphere   float64 0.0
  * isobaricInhPa      (isobaricInhPa) float64 1e+03 925.0 850.0 ... 100.0 50.0
  * step_bins          (step_bins) object (0 days 00:00:00, 1 days 00:00:00] ...
Data variables:
    u10                (step_bins, latitude, longitude) float32 3.894 ... 4.934
    v10                (step_bins, latitude, longitude) float32 -1.217 ... 2.021
    sp                 (step_bins, latitude, longitude) float32 1.015e+05 ......
    msl                (step_bins, latitude, longitude) float32 1.018e+05 ......
    tcwv               (step_bins, latitude, longitude) float32 0.5399 ... -2...
    u                  (step_bins, isobaricInhPa, latitude, longitude) float32 ...
    v                  (step_bins, isobaricInhPa, latitude, longitude) float32 ...
    z                  (step_bins, isobaricInhPa, latitude, longitude) float32 ...
    t                  (step_bins, isobaricInhPa, latitude, longitude) float32 ...
    r                  (step_bins, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-11-17T14:48 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
file = "SFNO_HalfYearForecast_20231116-1441.grib"
ds_sfno_halfyear = xr.open_dataset(os.path.join(path,file), engine="cfgrib")

skipping variable: paramId==228246 shortName='u100'
Traceback (most recent call last):
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=10.0) new_value=Variable(dimensions=(), data=100.0)
skipping variable: paramId==228247 shortName='v100'
Traceback (most recent call last):
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/lenny/miniconda3/envs/weather/lib/python3.12/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.data